In [19]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [20]:
base = pd.read_csv('titanic_train.csv')

In [21]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [22]:
base = base.drop('Cabin',axis=1)
colunas = ['Name','Ticket','PassengerId']
base = base.drop(colunas,axis=1)
base['IsMale'] = (base['Sex'] == 'male').astype(int)
base.drop('Sex',axis=1,inplace=True)
Embarked_dummies = pd.get_dummies(base.Embarked).astype(int)
base = pd.concat([base,Embarked_dummies],axis=1)
base.drop('Embarked',axis=1,inplace=True)
base['Age'] = base['Age'].fillna(base.groupby(['Pclass'])['Age'].transform('mean'))
base = base.drop(['C','Q','S'], axis=1)
base.info()
base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   IsMale    891 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 48.9 KB


,Survived,Pclass,Age,SibSp,Parch,Fare,IsMale
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1


In [23]:
X = base.drop('Survived', axis=1)
y = base.Survived

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score


modelos = {
    "Regressao": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier()
}

scaler = StandardScaler()
X_result = scaler.fit_transform(X)

def treinar_e_avaliar_modelo(modelos,X, y):
    for nome, modelo in modelos.items():
        modelo.fit(X,y)
        print(nome, modelo.score(X,y))


treinar_e_avaliar_modelo(modelos,X_result,y)


Regressao 0.8035914702581369
KNN 0.8608305274971941
RandomForest 0.9820426487093153


##Importando a base de teste e tratando os dados

In [25]:
base_test = pd.read_csv('titanic_test.csv')
base_test = base_test.drop('Cabin',axis=1)
colunas = ['Name','Ticket','PassengerId']
base_test = base_test.drop(colunas,axis=1)
base_test['IsMale'] = (base_test['Sex'] == 'male').astype(int)
base_test.drop('Sex',axis=1,inplace=True)
Embarked_dummies = pd.get_dummies(base_test.Embarked).astype(int)
base_test = pd.concat([base_test,Embarked_dummies],axis=1)
base_test.drop('Embarked',axis=1,inplace=True)
base_test['Age'] = base_test['Age'].fillna(base_test.groupby(['Pclass'])['Age'].transform('mean'))
base_test = base_test.dropna()
base_test = base_test.drop(['C','Q','S',], axis=1)
base_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417 entries, 0 to 417
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  417 non-null    int64  
 1   Pclass    417 non-null    int64  
 2   Age       417 non-null    float64
 3   SibSp     417 non-null    int64  
 4   Parch     417 non-null    int64  
 5   Fare      417 non-null    float64
 6   IsMale    417 non-null    int64  
dtypes: float64(2), int64(5)
memory usage: 26.1 KB


In [26]:
X_teste = base_test.drop('Survived', axis=1)
y_teste = base_test.Survived

In [27]:
X_teste_result = scaler.transform(X_teste)

def avaliar_modelo(modelos, X, y):
    preds = {}  # Dicionário para guardar previsões
    for nome, modelo in modelos.items():
        y_pred = modelo.predict(X)

        acc = accuracy_score(y, y_pred)
        prec = precision_score(y, y_pred)
        rec = recall_score(y, y_pred)

        print(f"Modelo: {nome}")
        print(f"  Acurácia:  {acc:.4f}")
        print(f"  Precisão:  {prec:.4f}")
        print(f"  Recall:    {rec:.4f}")
        print("-" * 30)

        preds[nome] = y_pred  # Guarda predições

    return preds

# Chama a função e captura as previsões
predicoes = avaliar_modelo(modelos, X_teste_result, y_teste)

# Agora, para o RandomForest, adiciona a coluna no DataFrame
base_test['check_Survived'] = predicoes['Regressao']

# Confirma visualizando
base_test[['Survived', 'check_Survived']].head(30)

Modelo: Regressao
  Acurácia:  0.9472
  Precisão:  0.9114
  Recall:    0.9474
------------------------------
Modelo: KNN
  Acurácia:  0.8082
  Precisão:  0.7368
  Recall:    0.7368
------------------------------
Modelo: RandomForest
  Acurácia:  0.8273
  Precisão:  0.7703
  Recall:    0.7500
------------------------------


,Survived,check_Survived
0,0,0
1,1,0
2,0,0
3,0,0
4,1,1
5,0,0
6,1,1
7,0,0
8,1,1
9,0,0
